# 缓存二进制数据

除了缓存文本数据外，Redis 还经常被用于缓存二进制数据，如图片、视频、音频等。

Redis 存储二进制时比较特别，只需要在初始化 redis-py 客户端实例的时候让 decode-responses 为 False，也就是让返回的数据**不自动进行解码**，而是以二进制形式返回。如果存储图片后，不将 decode_responses 设置为 False，获取二进制数据的时候 redis-py 客户端会把它们解释为文本数据，会引发错误。

需要注意，没有办法单独对某个 GET 操作设置 decode-responses 为 False，所以如果 client 还有别的操作需要进行自动解码，只能新声明一个 client 来进行对二进制的操作。

In [14]:
"""
配置连接
"""
from redis import Redis

# Redis连接配置
client = Redis(
    host='39.104.208.122', 
    port=6379,
    decode_responses=False,  # 关闭自动解码
    ssl=False
)

if client.ping():
    print("Redis连接成功")
else:
    print("Redis连接失败")

Redis连接成功


In [15]:
class BinaryCache:

    def __init__(self, client):
        self.client = client

    def set(self, name, path, ttl=None):
        """
        根据给定的名字和文件路径，缓存指定的二进制文件数据。
        可选ttl参数用于设置缓存的生存时间。
        """
        # 以二进制方式打开文件，并读取文件中的数据
        file = open(path, "rb")
        data = file.read()
        file.close()
        # 缓存二进制数据
        self.client.set(name, data, ttl)

    def get(self, name):
        """
        获取指定名字的缓存内容，若缓存不存在则返回None。
        """
        return self.client.get(name)

In [16]:
cache = BinaryCache(client)
cache.set("logo.png", "./logo.png")
print(cache.get("logo.png")[:10])

b'\x89PNG\r\n\x1a\n\x00\x00'


In [13]:
# 如果想看到照片，可以使用第三方库进行转换；或者用下面的方式将二进制保存为图片
with open("new_logo.png", "wb") as f:
    f.write(cache.get("logo.png"))

多说一嘴，我记得以前存图片还要用什么 Base64 进行转换，Base64 是干嘛的？

Base64 编码是从二进制值转换到某些特定字符的编码方式，这些特定字符一共 64 个，所以称作 Base64。为什么不直接传输二进制呢？而且即使 Base64 编码过的字符串最终也是二进制在网络上传输的。原因其实是二进制不兼容的问题。某些二进制值，在一些硬件上，比如在不同的路由器，老电脑上，表示的意义不一样，做的处理也不一样。但是 Base64 的 64 个字符（A-Z、a-z、0-9、+、/）是 ASCII 字符，在 ASCII 和 UTF-8 中编码一致，因此在大多数机器和软件上的行为相同，兼容性强。

它的过程：
+ 图片的二进制数据 → Base64 编码 → ASCII 字符串。
+ ASCII 字符串 → Base64 解码 → 二进制数据 → 保存或显示为图片。

另外说一下，如果在 MySQL 中存储二进制，需要用到 BINARY，用于存储固定长度或可变长度的二进制数据；BLOB，适合存储大块二进制数据。存储的过程和 Redis 也是一样的，read 拿到二进制直接存。